In [1]:
from collections import abc
import keyword

In [2]:
class FrozenJSON:
    """A read-only façade for navigating a JSON-like object
       using attribute notation
    """

# BEGIN EXPLORE1
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if keyword.iskeyword(key):  # <1>
                key += '_'
            self.__data[key] = value
# END EXPLORE1

    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:  # <8>
            return obj

#### explore1.py: Script to explore the OSCON schedule feed

In [3]:
import import_ipynb
from osconfeed import load

importing Jupyter notebook from osconfeed.ipynb
  1 conferences
484 events
357 speakers
 53 venues


In [4]:
>>> raw_feed = load()
>>> feed = FrozenJSON(raw_feed)
>>> len(feed.Schedule.speakers)

357

In [5]:
>>> sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [6]:
>>> for key, value in sorted(feed.Schedule.items()):
...     print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [7]:
>>> feed.Schedule.speakers[-1].name

'Carina C. Zona'

In [8]:
>>> talk = feed.Schedule.events[40]
>>> type(talk)

__main__.FrozenJSON

In [9]:
>>> talk.name

'There *Will* Be Bugs'

In [10]:
>>> talk.speakers

[3471, 5199]

In [11]:
>>> talk.flavor

KeyError: 'flavor'

#### Handle keywords by appending a `_`.

In [12]:
>>> grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})
>>> grad.name

'Jim Bo'

In [13]:
>>> grad.class_

1982